In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [22]:
train = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
train

In [23]:
test = pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')
test

In [24]:
all_data = pd.concat([train, test])
all_data

In [25]:
all_data['Dates'] = pd.to_datetime(all_data['Dates'])
all_data['Year'] = all_data['Dates'].dt.year
all_data['Month'] = all_data['Dates'].dt.month
all_data['Day'] = all_data['Dates'].dt.day
all_data['Hour'] = all_data['Dates'].dt.hour
all_data['Minute'] = all_data['Dates'].dt.minute

In [26]:
all_data_2 = all_data.drop(columns=['Dates','Category','Descript','Resolution','Id'])
all_data_2

In [27]:
from sklearn.preprocessing import LabelEncoder

In [28]:
le = LabelEncoder()
cat_cols = all_data_2.columns[all_data_2.dtypes == object]
cat_cols

In [29]:
for col in cat_cols:
    all_data_2[col] = le.fit_transform(all_data_2[col])

In [30]:
train_2 = all_data_2[:len(train)]
test_2 = all_data_2[len(train):]

In [31]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(train_2, train['Category'], test_size=0.2, random_state=42, stratify=train['Category'])
x_val

In [32]:
from catboost import CatBoostClassifier

cbc = CatBoostClassifier(verbose=50, task_type='GPU', learning_rate=0.203863, iterations=10000)
cbc.fit(x_train, y_train, eval_set=(x_val, y_val), early_stopping_rounds=25)

In [33]:
result = cbc.predict_proba(test_2)

In [34]:
result.shape

In [35]:
sub = pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip')
sub

In [36]:
sub.iloc[:,1:] = result
sub

In [37]:
sub.to_csv('sub.csv',index=False)